# CREATING DATAFRAME

In [2]:
import numpy as np
import os
import conf as config
import creating_DF as cdf
import cv2
import random
import glob
from collections import Counter
from random import sample,seed
seed(10)

We need to make sure we have pictures of ourselves and from random people in all of our folders in a balanced way (positive, negative and anchor).

⚠️⚠️⚠️ `This code was ran once with some variations before the folders were correctly filled with the desired images, you only need to run the last cells`

**We create lists with all the people from LFW dataset**

In [3]:
dir_list = os.listdir(config.NEG_PATH)
neg_list = sorted(dir_list)

**We get the names and we look for those people that appears more than 10 times**

In [4]:
name = [person[:-9] for person in neg_list]
freq_person = Counter(name)
pairs = freq_person.items()
filtered_people = {key: value for key, value in pairs if value > 9}

In [5]:
filtered_people 

{'Abdullah_Gul': 19,
 'Adrien_Brody': 12,
 'Alejandro_Toledo': 39,
 'Alvaro_Uribe': 35,
 'Amelie_Mauresmo': 21,
 'Andre_Agassi': 36,
 'Andy_Roddick': 15,
 'Angelina_Jolie': 20,
 'Ann_Veneman': 11,
 'Anna_Kournikova': 12,
 'Ari_Fleischer': 13,
 'Ariel_Sharon': 77,
 'Arnold_Schwarzenegger': 42,
 'Atal_Bihari_Vajpayee': 24,
 'Bill_Clinton': 29,
 'Bill_Gates': 17,
 'Bill_McBride': 10,
 'Bill_Simon': 15,
 'Britney_Spears': 14,
 'Carlos_Menem': 21,
 'Carlos_Moya': 19,
 'Catherine_Zeta-Jones': 11,
 'Charles_Moose': 13,
 'Colin_Powell': 236,
 'Condoleezza_Rice': 11,
 'David_Beckham': 31,
 'David_Nalbandian': 14,
 'Dick_Cheney': 14,
 'Dominique_de_Villepin': 15,
 'Donald_Rumsfeld': 121,
 'Edmund_Stoiber': 13,
 'Eduardo_Duhalde': 14,
 'Fidel_Castro': 18,
 'George_HW_Bush': 13,
 'George_Robertson': 22,
 'George_W_Bush': 80,
 'Gerhard_Schroeder': 109,
 'Gloria_Macapagal_Arroyo': 44,
 'Gonzalo_Sanchez_de_Lozada': 12,
 'Gordon_Brown': 13,
 'Gray_Davis': 26,
 'Guillermo_Coria': 30,
 'Halle_Berry': 16

In [6]:
for key in filtered_people.keys():
    if key != 'Julieta_Staryfurman' and key != 'Sahar_Garber' and key != 'Noam_Pinhas_Cohen':
        filtered_people[key] = filtered_people[key] // 2

In [7]:
#nr_rows = 3 * (25**2)
nr_rows = 0
for value in filtered_people.values():
    nr_rows += (value**2)


In [8]:
len(filtered_people)

161

In [9]:
nr_rows

56434

In [10]:
filtered_people

{'Abdullah_Gul': 9,
 'Adrien_Brody': 6,
 'Alejandro_Toledo': 19,
 'Alvaro_Uribe': 17,
 'Amelie_Mauresmo': 10,
 'Andre_Agassi': 18,
 'Andy_Roddick': 7,
 'Angelina_Jolie': 10,
 'Ann_Veneman': 5,
 'Anna_Kournikova': 6,
 'Ari_Fleischer': 6,
 'Ariel_Sharon': 38,
 'Arnold_Schwarzenegger': 21,
 'Atal_Bihari_Vajpayee': 12,
 'Bill_Clinton': 14,
 'Bill_Gates': 8,
 'Bill_McBride': 5,
 'Bill_Simon': 7,
 'Britney_Spears': 7,
 'Carlos_Menem': 10,
 'Carlos_Moya': 9,
 'Catherine_Zeta-Jones': 5,
 'Charles_Moose': 6,
 'Colin_Powell': 118,
 'Condoleezza_Rice': 5,
 'David_Beckham': 15,
 'David_Nalbandian': 7,
 'Dick_Cheney': 7,
 'Dominique_de_Villepin': 7,
 'Donald_Rumsfeld': 60,
 'Edmund_Stoiber': 6,
 'Eduardo_Duhalde': 7,
 'Fidel_Castro': 9,
 'George_HW_Bush': 6,
 'George_Robertson': 11,
 'George_W_Bush': 40,
 'Gerhard_Schroeder': 54,
 'Gloria_Macapagal_Arroyo': 22,
 'Gonzalo_Sanchez_de_Lozada': 6,
 'Gordon_Brown': 6,
 'Gray_Davis': 13,
 'Guillermo_Coria': 15,
 'Halle_Berry': 8,
 'Hamid_Karzai': 11,
 'H

**We save half of the pictures of those people that appears more than 10 times in the anchor folder and the other half in the positive file**

In [ ]:
new_path_anchor = []
new_path_positive = []
for k,v in filtered_people.items():
    for i in range(1,v//2+1):
        new_path_anchor.append(k + '_' + '0'*(4-len(str(i))) + str(i) +'.jpg')
    for i in range(v//2+1,((v//2)*2)+1):
        new_path_positive.append(k + '_' + '0'*(4-len(str(i))) + str(i) +'.jpg')

In [ ]:
len(new_path_anchor) == len(new_path_positive) 

**We changed the name of the pictures of ourselves so that it follows the same logic as in the lfw dataset.**

In [ ]:
path = '/Users/julietastaryfurmanshalom/OneDrive/ITC/Face_Rec/sahar/anchor'
name = 'Sahar_Garber'

In [ ]:
cdf.changing_names(path,name)

**We make sure we have all the images in all of the folders**

In [ ]:
src_path = '/Users/julietastaryfurmanshalom/OneDrive/ITC/Face_Rec/data/negative/'
dest_path = '/Users/julietastaryfurmanshalom/OneDrive/ITC/Face_Rec/data/anchor/'

In [ ]:
cdf.add_to_folder(new_path_anchor, src_path, dest_path)

In [ ]:
pos_images = os.listdir(config.POS_PATH)
anchor_images = os.listdir(config.ANC_PATH)
all_images = pos_images + anchor_images

In [ ]:
src_path = '/Users/julietastaryfurmanshalom/OneDrive/ITC/Face_Rec/data/positive/'
dest_path = '/Users/julietastaryfurmanshalom/OneDrive/ITC/Face_Rec/data/negative/'

In [ ]:
sample_all_images = sample(list(set(all_images)),len(pos_images))

In [ ]:
cdf.add_to_folder(pos_images, src_path, dest_path)

In [ ]:
src_path = '/Users/julietastaryfurmanshalom/OneDrive/ITC/Face_Rec/data/negative/'
dest_path = '/Users/julietastaryfurmanshalom/OneDrive/ITC/Face_Rec/data/negative_cleaned/'

In [ ]:
add_to_folder(sample_all_images, src_path, dest_path)

🚀🚀🚀**We create the dataset. This are the only cells you should run!!!**

In [13]:
def rescale_resize(img):
    """
    This function rescales a given image
    :param img: image to resize
    :return: resized image
    """
    return cv2.resize(img, (105, 105)) / 255

In [14]:
def load_from_path(path,nr_rows,filtered_people):
    """
    This function loads from a given path of a folder, all the content to an array
    :param path: the path of the folder
    :param img_num: number of images in the folder
    :return: np array with the content of the folder
    """
    imgs = []
    dir_list = os.listdir(path)
    
    if 'negative_cleaned' in path:
        for i in range(nr_rows):
            sample = random.sample(dir_list,1)[0]
            img = cv2.imread(os.path.join(path, sample))
            img = rescale_resize(img)
            imgs.append(img)
        
    
    
    else:
        dir_list = sorted(dir_list)

        img_num = len(dir_list)
        for i in range(img_num):
            if 'anchor' in path:
                img = cv2.imread(os.path.join(path, dir_list[i]))
                img = rescale_resize(img)       
                for _ in range(filtered_people[dir_list[i][:-9]]):
                    imgs.append(img)

            elif 'positive' in path:
                name = dir_list[i][:-9]
                if name != "":
                    files = []
                    for name in glob.glob(path + name + '*'):
                        files.append(name)
                    files = sorted(files)
                    for file in files:
                        if file != "":
                            img = cv2.imread(file)
                            img = rescale_resize(img)
                            imgs.append(img)


   
    return np.array(imgs)

In [ ]:
#pos = load_from_path('/Users/julietastaryfurmanshalom/OneDrive/ITC/Face_Rec/data/positive/',nr_rows)

In [ ]:
#pos.shape

In [ ]:
#pos.shape[0] - nr_rows 

In [15]:
def load_data():
    """
    This function loads the data to three np.array (anchor, positive, negative)
    :param img_num: number of images to load (by default 2271)
    :return: a tuple with the np arrays X and y
    """
    anchor = load_from_path('/Users/julietastaryfurmanshalom/OneDrive/ITC/Face_Rec/data/anchor/',56434,filtered_people )
    print(anchor.shape)
    positive = load_from_path('/Users/julietastaryfurmanshalom/OneDrive/ITC/Face_Rec/data/positive/',56434,filtered_people )
    print(positive.shape)
    negative = load_from_path('/Users/julietastaryfurmanshalom/OneDrive/ITC/Face_Rec/data/negative_cleaned/',56434,filtered_people )
    print(negative.shape)
    return (anchor, positive, negative)

In [16]:
anchor,positive,negative = load_data()

(56434, 105, 105, 3)
(56434, 105, 105, 3)
(56434, 105, 105, 3)


In [23]:
len(anchor[0:128])

128

In [30]:
def create_batches(anchor, positive, negative):
    nr_batch = 0
    for i in range(0,len(anchor), 128):
        top = min(i + 128, len(anchor))
        X = np.concatenate([np.array([anchor[i:top], positive[i:top]]), np.array([anchor[i:top], negative[i:top]])], axis=1)
        y = np.concatenate([np.ones(128), np.zeros(128)], axis=0) 
        nr_batch += 1
        np.save("dataset/X_batch_" + str(nr_batch) , X)
        np.save("dataset/y_batch_" + str(nr_batch) , y)

In [31]:
create_batches(anchor, positive, negative)